In [1]:
import os
import sys
import numpy as np
import pandas as pd
from geopy import Point
from geopy import distance
pd.set_option('display.max_columns', 999)

In [2]:
# load datasets
rw = pd.read_csv("..//Datasets/NEW/RW_flights.csv", parse_dates=['date'])
rw_info = pd.read_csv("..//Datasets/NEW/RW_services_info.csv")
airports = pd.read_csv("..//Datasets/Airports/BTS_Airports_LAT_LON.csv")
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [3]:
# number of existing tail numbers with flight history
rw.tail_number.nunique() 

899

In [99]:
# number of unique air medical services
rw.Owner.nunique()

215

In [4]:
# The percentage of the total tail numbers
print(round(rw.tail_number.nunique() / rw_info.Tail_number.nunique() * 100, 2), "%")

82.78 %


In [5]:
# count of training flights 
len(rw.loc[rw.origin == rw.destination])

38107

In [6]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136391 entries, 0 to 136390
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   tail_number           136391 non-null  object        
 1   date                  136391 non-null  datetime64[ns]
 2   aircraft              136391 non-null  object        
 3   origin                136391 non-null  object        
 4   origin_location       136391 non-null  object        
 5   destination           136314 non-null  object        
 6   destination_location  136391 non-null  object        
 7   departure             136391 non-null  object        
 8   arrival               136186 non-null  object        
 9   duration              136331 non-null  object        
dtypes: datetime64[ns](1), object(9)
memory usage: 10.4+ MB


In [7]:
rw.isna().sum()

tail_number               0
date                      0
aircraft                  0
origin                    0
origin_location           0
destination              77
destination_location      0
departure                 0
arrival                 205
duration                 60
dtype: int64

#### Data preprocessing
---

In [8]:
# check duplicates airport_codes
airport_codes.duplicated(subset=['gps_code']).sum()

16578

In [9]:
# check duplicates airport_codes
airport_codes.duplicated(subset=['local_code']).sum()

28768

In [10]:
# Remove useless words from rw
rw = rw.replace(regex={'Near ':'', 'First seen ':'', 'Last seen ':'', 'result unknown': np.nan, 'En Route':None, 'unknown': np.nan,
                       'n/a': np.nan, 'Unknown':None, 'Diverted':np.nan, '¬†':' ', 'Scheduled': None, 'Cancelled': np.nan})

In [11]:
# count of incomplete flights 
len(rw.loc[(rw.arrival.isna()) | (rw.duration.isna())])

270

In [12]:
# split origin_Latitude and origin_Longitude
rw['origin_Latitude'] = rw[rw.origin_location.str.startswith('L ')].origin_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[0]
rw['origin_Longitude'] = rw[rw.origin_location.str.startswith('L ')].origin_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[1]

In [13]:
# split destination_Latitude and destination_Longitude
rw['destination_Latitude'] = rw[rw.destination_location.str.startswith('L ', na=False)].destination_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[0]
rw['destination_Longitude'] = rw[rw.destination_location.str.startswith('L ', na=False)].destination_location.replace(regex={'L ':''}).str.split(" ", n = 1, expand = True)[1]

In [14]:
# convert to numeric
rw['origin_Latitude'] = pd.to_numeric(rw.origin_Latitude, errors='coerce')
rw['origin_Longitude'] = pd.to_numeric(rw.origin_Longitude, errors='coerce')

rw['destination_Latitude'] = pd.to_numeric(rw.destination_Latitude, errors='coerce')
rw['destination_Longitude'] = pd.to_numeric(rw.destination_Longitude, errors='coerce')

In [15]:
# split ICAO_code
rw['ICAO_code_origin'] = rw[rw.origin_Latitude.isna()].origin_location.str.rsplit(" - ", n = 1, expand = True)[1]
rw['ICAO_code_destination'] = rw[rw.destination_Latitude.isna()].destination_location.str.rsplit(" - ", n = 1, expand = True)[1]

In [16]:
# replace dual ICAO code
rw.loc[rw.ICAO_code_origin.str.contains(' / ', na=False), 'ICAO_code_origin'] = rw.loc[rw.ICAO_code_origin.str.contains(' / ', na=False), 'ICAO_code_origin'].str.split(" / ", expand=True, n=1)[0]

rw.loc[rw.ICAO_code_destination.str.contains(' / ', na=False), 'ICAO_code_destination'] = rw.loc[rw.ICAO_code_destination.str.contains(' / ', na=False), 'ICAO_code_destination'].str.split(" / ", expand=True, n=1)[0]

#### Origin Location
---

In [17]:
# merge origin airport information 1 time
rw = rw.merge(airport_codes[['ident']].add_suffix('_origin_1'),
         how='left', left_on='ICAO_code_origin', right_on='ident_origin_1')

In [18]:
# Switch dual ICAO codes
rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()) & rw.origin.str.contains(' / '), 'ICAO_code_origin'] = rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()) & rw.origin.str.contains(' / '), 'origin'].str.split(' / ', n=1, expand=True)[1].str.replace(')', '')

In [19]:
# merge origin airport information 2 time
rw = rw.merge(airport_codes[['ident']].add_suffix('_origin_2'),
         how='left', left_on='ICAO_code_origin', right_on='ident_origin_2')

In [20]:
# merge similar columns
rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()) & (rw.origin.str.contains(' / ')), 'ident_origin_1'] = rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()) & (rw.origin.str.contains(' / ')), 'ident_origin_2']

In [21]:
# Drop duplicated column
rw = rw.drop(['ident_origin_2'], axis=1)

In [22]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [23]:
# drop duplicates in gps_code
airport_codes = airport_codes.drop_duplicates(subset=['gps_code'])

In [24]:
# merge origin airport information 3 time
rw = rw.merge(airport_codes[['gps_code', 'ident']].add_suffix('_origin_2'),
         how='left', left_on='ICAO_code_origin', right_on='gps_code_origin_2')

In [25]:
# merge similar columns
rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()), 'ident_origin_1'] = rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()), 'ident_origin_2']

In [26]:
# Drop duplicated columns
rw = rw.drop(['ident_origin_2', 'gps_code_origin_2'], axis=1)

In [27]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [28]:
# drop duplicates in local_code
airport_codes = airport_codes.drop_duplicates(subset=['local_code'])

In [29]:
# merge origin airport information 4 times
rw = rw.merge(airport_codes[['local_code', 'ident']].add_suffix('_origin_2'),
         how='left', left_on='ICAO_code_origin', right_on='local_code_origin_2')

In [30]:
# merge similar columns
rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()), 'ident_origin_1'] = rw.loc[(rw.ICAO_code_origin.notnull()) & (rw.ident_origin_1.isna()), 'ident_origin_2']

In [31]:
# Drop duplicated columns
rw = rw.drop(['ident_origin_2', 'local_code_origin_2'], axis=1)

In [32]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [33]:
# split coordinates and convert to numeric
airport_codes["Latitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[1])
airport_codes["Longitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[0])

In [34]:
airport_codes = airport_codes.replace(regex={r' Heliport': '', r' Airpark':'', r' Airport': '', r' Regional':'', r' Municipal':'', r' International':'',
                                            r' LRRS':''})

In [35]:
# add state to the name
airport_codes.loc[:, 'name'] = airport_codes.loc[:, 'name'] + ", " + airport_codes.iso_region.str.split('-', expand=True)[1]

In [36]:
# drop duplicates
airport_codes = airport_codes.drop_duplicates(subset=['name'])

In [37]:
# merge origin airport information 5 times
rw = rw.merge(airport_codes[['name', 'ident']].add_suffix('_origin_2'),
         how='left', left_on='origin', right_on='name_origin_2')

In [38]:
# merge similar columns
rw.loc[(rw.ICAO_code_origin.isna()), 'ident_origin_1'] = rw.loc[(rw.ICAO_code_origin.isna()), 'ident_origin_2'] 
rw.loc[(rw.ICAO_code_origin.isna()), 'ICAO_code_origin'] = rw.loc[(rw.ICAO_code_origin.isna()), 'ident_origin_2'] 

In [39]:
# Drop duplicated columns
rw = rw.drop(['ident_origin_2', 'name_origin_2'], axis=1)

In [40]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

# split coordinates and convert to numeric
airport_codes["Latitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[1])
airport_codes["Longitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[0])

# remove all extra words
airport_codes = airport_codes.replace(regex={r' Heliport': '', r' Airpark':'', r' Airport': '', r' Regional':'', r' Municipal':'', r' International':'',
                                            r' LRRS':''})
# add state to municipality
airport_codes.loc[:, 'municipality'] = airport_codes.loc[:, 'municipality'] + ", " + airport_codes.iso_region.str.split('-', expand=True)[1]

# drop duplicates
airport_codes = airport_codes.drop_duplicates(subset=['municipality'])

In [41]:
# merge origin airport information 6 times
rw = rw.merge(airport_codes[['municipality', 'ident']].add_suffix('_origin_2'),
         how='left', left_on='origin', right_on='municipality_origin_2')

In [42]:
# merge similar columns
rw.loc[(rw.ICAO_code_origin.isna()), 'ident_origin_1'] = rw.loc[(rw.ICAO_code_origin.isna()), 'ident_origin_2'] 
rw.loc[(rw.ICAO_code_origin.isna()), 'ICAO_code_origin'] = rw.loc[(rw.ICAO_code_origin.isna()), 'ident_origin_2'] 

In [43]:
# Drop duplicated columns
rw = rw.drop(['ident_origin_2', 'municipality_origin_2'], axis=1)

In [44]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136391 entries, 0 to 136390
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tail_number            136391 non-null  object        
 1   date                   136391 non-null  datetime64[ns]
 2   aircraft               128675 non-null  object        
 3   origin                 136391 non-null  object        
 4   origin_location        136391 non-null  object        
 5   destination            136314 non-null  object        
 6   destination_location   136314 non-null  object        
 7   departure              136391 non-null  object        
 8   arrival                136175 non-null  object        
 9   duration               136121 non-null  object        
 10  origin_Latitude        60973 non-null   float64       
 11  origin_Longitude       60973 non-null   float64       
 12  destination_Latitude   53537 non-null   floa

In [45]:
rw.isna().sum()

tail_number                  0
date                         0
aircraft                  7716
origin                       0
origin_location              0
destination                 77
destination_location        77
departure                    0
arrival                    216
duration                   270
origin_Latitude          75418
origin_Longitude         75418
destination_Latitude     82854
destination_Longitude    82854
ICAO_code_origin           500
ICAO_code_destination    53615
ident_origin_1            1047
dtype: int64

#### Destination Location 
---

In [46]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [47]:
# merge destination airport information 1 time
rw = rw.merge(airport_codes[['ident']].add_suffix('_destination_1'),
         how='left', left_on='ICAO_code_destination', right_on='ident_destination_1')

In [48]:
# Switch dual ICAO codes
rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()) & rw.destination.str.contains(' / '), 'ICAO_code_destination'] = rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()) & rw.destination.str.contains(' / '), 'destination'].str.split(' / ', n=1, expand=True)[1].str.replace(')', '')

In [49]:
# merge destination airport information 2 time
rw = rw.merge(airport_codes[['ident']].add_suffix('_destination_2'),
         how='left', left_on='ICAO_code_destination', right_on='ident_destination_2')

In [50]:
# merge similar columns
rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()) & (rw.destination.str.contains(' / ')), 'ident_destination_1'] = rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()) & (rw.destination.str.contains(' / ')), 'ident_destination_2']

In [51]:
# Drop duplicated column
rw = rw.drop(['ident_destination_2'], axis=1)

In [52]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [53]:
# drop duplicates in gps_code
airport_codes = airport_codes.drop_duplicates(subset=['gps_code'])

In [54]:
# merge destination airport information 3 time
rw = rw.merge(airport_codes[['gps_code', 'ident']].add_suffix('_destination_2'),
         how='left', left_on='ICAO_code_destination', right_on='gps_code_destination_2')

In [55]:
# merge similar columns
rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()), 'ident_destination_1'] = rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()), 'ident_destination_2']

In [56]:
# Drop duplicated columns
rw = rw.drop(['ident_destination_2', 'gps_code_destination_2'], axis=1)

In [57]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [58]:
# drop duplicates in local_code
airport_codes = airport_codes.drop_duplicates(subset=['local_code'])

In [59]:
# merge destination airport information 4 times
rw = rw.merge(airport_codes[['local_code', 'ident']].add_suffix('_destination_2'),
         how='left', left_on='ICAO_code_destination', right_on='local_code_destination_2')

In [60]:
# merge similar columns
rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()), 'ident_destination_1'] = rw.loc[(rw.ICAO_code_destination.notnull()) & (rw.ident_destination_1.isna()), 'ident_destination_2']

In [61]:
# Drop duplicated columns
rw = rw.drop(['ident_destination_2', 'local_code_destination_2'], axis=1)

In [62]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [63]:
# split coordinates and convert to numeric
airport_codes["Latitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[1])
airport_codes["Longitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[0])

In [64]:
airport_codes = airport_codes.replace(regex={r' Heliport': '', r' Airpark':'', r' Airport': '', r' Regional':'', 
                                             r' Municipal':'', r' International':'', r' LRRS':''})

In [65]:
airport_codes.loc[:, 'name'] = airport_codes.loc[:, 'name'] + ", " + airport_codes.iso_region.str.split('-', expand=True)[1]
airport_codes.loc[:, 'municipality'] = airport_codes.loc[:, 'municipality'] + ", " + airport_codes.iso_region.str.split('-', expand=True)[1]

In [66]:
# drop duplicates
airport_codes = airport_codes.drop_duplicates(subset=['municipality'])
airport_codes = airport_codes.drop_duplicates(subset=['name'])

In [67]:
# merge destination airport information 5 times
rw = rw.merge(airport_codes[['name', 'ident']].add_suffix('_destination_2'),
         how='left', left_on='destination', right_on='name_destination_2')

In [68]:
# merge similar columns
rw.loc[(rw.ICAO_code_destination.isna()), 'ident_destination_1'] = rw.loc[(rw.ICAO_code_destination.isna()), 'ident_destination_2'] 
rw.loc[(rw.ICAO_code_destination.isna()), 'ICAO_code_destination'] = rw.loc[(rw.ICAO_code_destination.isna()), 'ident_destination_2'] 

In [69]:
# Drop duplicated columns
rw = rw.drop(['ident_destination_2', 'name_destination_2'], axis=1)

In [70]:
# merge destination airport information 6 times
rw = rw.merge(airport_codes[['municipality', 'ident']].add_suffix('_destination_2'),
         how='left', left_on='destination', right_on='municipality_destination_2')

In [71]:
# merge similar columns
rw.loc[(rw.ICAO_code_destination.isna()), 'ident_destination_1'] = rw.loc[(rw.ICAO_code_destination.isna()), 'ident_destination_2'] 
rw.loc[(rw.ICAO_code_destination.isna()), 'ICAO_code_destination'] = rw.loc[(rw.ICAO_code_destination.isna()), 'ident_destination_2'] 

In [72]:
# Drop duplicated columns
rw = rw.drop(['ident_destination_2', 'municipality_destination_2'], axis=1)

In [73]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136391 entries, 0 to 136390
Data columns (total 18 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tail_number            136391 non-null  object        
 1   date                   136391 non-null  datetime64[ns]
 2   aircraft               128675 non-null  object        
 3   origin                 136391 non-null  object        
 4   origin_location        136391 non-null  object        
 5   destination            136314 non-null  object        
 6   destination_location   136314 non-null  object        
 7   departure              136391 non-null  object        
 8   arrival                136175 non-null  object        
 9   duration               136121 non-null  object        
 10  origin_Latitude        60973 non-null   float64       
 11  origin_Longitude       60973 non-null   float64       
 12  destination_Latitude   53537 non-null   floa

In [74]:
rw.isna().sum()

tail_number                  0
date                         0
aircraft                  7716
origin                       0
origin_location              0
destination                 77
destination_location        77
departure                    0
arrival                    216
duration                   270
origin_Latitude          75418
origin_Longitude         75418
destination_Latitude     82854
destination_Longitude    82854
ICAO_code_origin           500
ICAO_code_destination      597
ident_origin_1            1047
ident_destination_1       1083
dtype: int64

#### Coordinates merging
---

In [75]:
# load airport_codes again
airport_codes = pd.read_csv("..//Datasets/Airports/airport-codes.csv")

In [76]:
# split coordinates and convert to numeric
airport_codes["Latitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[1])
airport_codes["Longitude"] = pd.to_numeric(airport_codes.coordinates.str.split(', ', n=1, expand=True)[0])

In [77]:
# merge by origin
rw = rw.merge(airport_codes.add_suffix('_origin'),
         how='left', left_on='ident_origin_1', right_on='ident_origin')

In [78]:
# replace origin_Latitude and origin_Longitude
rw.loc[rw.origin_Latitude.isna(), 'origin_Latitude'] = rw.loc[rw.origin_Latitude.isna(), 'Latitude_origin']
rw.loc[rw.origin_Longitude.isna(), 'origin_Longitude'] = rw.loc[rw.origin_Longitude.isna(), 'Longitude_origin']

In [79]:
# drop all useless columns
rw = rw.drop(['coordinates_origin', 'Latitude_origin', 'Longitude_origin', 'continent_origin', 'elevation_ft_origin', 'type_origin',
         'local_code_origin', 'gps_code_origin', 'municipality_origin', 'iso_region_origin', 'ident_origin', 'iata_code_origin'], axis=1)

In [80]:
# merge by destination
rw = rw.merge(airport_codes.add_suffix('_destination'),
         how='left', left_on='ident_destination_1', right_on='ident_destination')

In [81]:
# replace destination_Latitude and destination_Longitude
rw.loc[rw.destination_Latitude.isna(), 'destination_Latitude'] = rw.loc[rw.destination_Latitude.isna(), 'Latitude_destination']
rw.loc[rw.destination_Longitude.isna(), 'destination_Longitude'] = rw.loc[rw.destination_Longitude.isna(), 'Longitude_destination']

In [82]:
# drop all useless columns
rw = rw.drop(['coordinates_destination', 'Latitude_destination', 'Longitude_destination', 'continent_destination', 'elevation_ft_destination', 'type_destination',
         'local_code_destination', 'gps_code_destination', 'municipality_destination', 'iso_region_destination', 'ident_destination', 'iata_code_destination'], axis=1)

In [83]:
rw.isna().sum()

tail_number                   0
date                          0
aircraft                   7716
origin                        0
origin_location               0
destination                  77
destination_location         77
departure                     0
arrival                     216
duration                    270
origin_Latitude             547
origin_Longitude            547
destination_Latitude        487
destination_Longitude       487
ICAO_code_origin            500
ICAO_code_destination       597
ident_origin_1             1047
ident_destination_1        1083
name_origin                1047
iso_country_origin         1047
name_destination           1083
iso_country_destination    1083
dtype: int64

In [84]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136391 entries, 0 to 136390
Data columns (total 22 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   tail_number              136391 non-null  object        
 1   date                     136391 non-null  datetime64[ns]
 2   aircraft                 128675 non-null  object        
 3   origin                   136391 non-null  object        
 4   origin_location          136391 non-null  object        
 5   destination              136314 non-null  object        
 6   destination_location     136314 non-null  object        
 7   departure                136391 non-null  object        
 8   arrival                  136175 non-null  object        
 9   duration                 136121 non-null  object        
 10  origin_Latitude          135844 non-null  float64       
 11  origin_Longitude         135844 non-null  float64       
 12  destination_Lati

#### Additional Information
---

In [85]:
# add Owner columns
rw = rw.merge(rw_info[['Tail_number','Owner']], how='left', left_on='tail_number', right_on='Tail_number').drop('Tail_number', axis=1)

In [86]:
# calculate the distance
for i in range(len(rw)):
    try:
        p1 = Point(str(rw.origin_Latitude[i]) + ' ' + str(rw.origin_Longitude[i]))
        p2 = Point(str(rw.destination_Latitude[i]) + ' ' + str(rw.destination_Longitude[i]))
    
        rw.loc[i, 'distance_mi'] = distance.distance(p1,p2).miles
    except:
        pass

In [87]:
# convert duration to minutes
rw.duration = pd.to_numeric(rw.duration.str.split(":", expand=True)[0]) * 60 + pd.to_numeric(rw.duration.str.split(":", expand=True)[1])

In [88]:
# split states 
rw['origin_state'] = rw.origin.str.split(', ', n=1, expand=True)[1]
rw.loc[rw.origin_state.isna(), 'origin_state'] = rw.origin_location.str.split(', ', n=1, expand=True)[1].str.rsplit(') ', n=1, expand=True)[0]
rw.loc[rw.origin_state.str.contains(' ', na=False), 'origin_state'] = rw.origin_location.str.rsplit(', ', n=1, expand=True)[1].str.split(')', n=1, expand=True)[0]

rw['destination_state'] = rw.destination.str.split(', ', n=1, expand=True)[1]
rw.loc[rw.destination_state.isna(), 'destination_state'] = rw.destination_location.str.split(', ', n=1, expand=True)[1].str.rsplit(') ', n=1, expand=True)[0]
rw.loc[rw.destination_state.str.contains(' ', na=False), 'destination_state'] = rw.destination_location.str.rsplit(', ', n=1, expand=True)[1].str.split(')', n=1, expand=True)[0]

In [89]:
# split cities
rw['city_origin'] = rw.origin_location.str.rsplit(" (", n=1, expand=True)[1].str.rsplit(", ", n=1, expand=True)[0]
rw.loc[rw.city_origin.isna(), 'city_origin'] = rw.loc[rw.origin.str.contains(', ', na=False), 'origin'].str.split(', ', expand=True)[0]
rw.loc[rw.city_origin.str.contains(' - ', na=False), 'city_origin'] = None

rw['city_destination'] = rw.destination_location.str.rsplit(" (", n=1, expand=True)[1].str.rsplit(", ", n=1, expand=True)[0]
rw.loc[rw.city_destination.isna(), 'city_destination'] = rw.loc[rw.destination.str.contains(', ', na=False), 'destination'].str.split(', ', expand=True)[0]
rw.loc[rw.city_destination.str.contains(' - ', na=False), 'city_origin'] = None

In [90]:
# add shift
rw['departure_shift'] = np.where((pd.to_datetime(rw.departure.str.split('M', n=1, expand=True)[0] + 'M', errors='coerce').dt.time.astype('str') < '19:00:00') &\
                       (pd.to_datetime(rw.departure.str.split('M', n=1, expand=True)[0] + 'M', errors='coerce').dt.time.astype('str') > '07:00:00'), 'day', 'night')

rw['arrival_shift'] = np.where((pd.to_datetime(rw.arrival.str.split('M', n=1, expand=True)[0] + 'M', errors='coerce').dt.time.astype('str') < '19:00:00') &\
                       (pd.to_datetime(rw.arrival.str.split('M', n=1, expand=True)[0] + 'M', errors='coerce').dt.time.astype('str') > '07:00:00'), 'day', 'night')

In [91]:
# rename columns and drop unnecessary columns
rw = rw.rename({'ident_origin_1': 'origin_code', 'ident_destination_1': 'destination_code', 
           'name_origin':'airport_origin', 'iso_country_origin': 'country_origin',
          'name_destination':'airport_destination', 'iso_country_destination': 'country_destination'}, axis=1)\
       .drop(['origin_location', 'destination_location', 'ICAO_code_origin', 'ICAO_code_destination'], axis=1)

In [92]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136391 entries, 0 to 136390
Data columns (total 26 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   tail_number            136391 non-null  object        
 1   date                   136391 non-null  datetime64[ns]
 2   aircraft               128675 non-null  object        
 3   origin                 136391 non-null  object        
 4   destination            136314 non-null  object        
 5   departure              136391 non-null  object        
 6   arrival                136175 non-null  object        
 7   duration               136121 non-null  float64       
 8   origin_Latitude        135844 non-null  float64       
 9   origin_Longitude       135844 non-null  float64       
 10  destination_Latitude   135904 non-null  float64       
 11  destination_Longitude  135904 non-null  float64       
 12  origin_code            135344 non-null  obje

In [93]:
# rearrange columns
rw = rw.iloc[:,:3].join([rw.iloc[:,12], rw.iloc[:,3], rw.iloc[:,13], rw.iloc[:,4], rw.iloc[:,5:8], rw.iloc[:,19], rw.iloc[:,18], 
                    rw.iloc[:,8:12], rw.iloc[:,14], rw.iloc[:,22], rw.iloc[:,20], rw.iloc[:,15],
                    rw.iloc[:,16], rw.iloc[:,23], rw.iloc[:,21], rw.iloc[:,17], rw.iloc[:,24:26]])

In [94]:
rw.head()

,tail_number,date,aircraft,origin_code,origin,destination_code,destination,departure,arrival,duration,distance_mi,Owner,origin_Latitude,origin_Longitude,destination_Latitude,destination_Longitude,airport_origin,city_origin,origin_state,country_origin,airport_destination,city_destination,destination_state,country_destination,departure_shift,arrival_shift
0,N640EC,2021-03-11,EC35,KPGV,Pitt-Greenville (KPGV),KRWI,Rocky Mount-Wilson Rgnl (KRWI),04:29AM EST,04:46AM EST,16.0,32.296705,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.635201,-77.385300,35.856300,-77.891899,Pitt Greenville Airport,Greenville,NC,US,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,night,night
1,N640EC,2021-03-11,EC35,KRWI,Rocky Mount-Wilson Rgnl (KRWI),KPGV,Pitt-Greenville (KPGV),02:17AM EST,03:47AM EST (?),89.0,32.296705,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.856300,-77.891899,35.635201,-77.385300,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,Pitt Greenville Airport,Greenville,NC,US,night,night
2,N640EC,2021-03-11,EC35,KPGV,Pitt-Greenville (KPGV),KRWI,Rocky Mount-Wilson Rgnl (KRWI),01:15AM EST,01:30AM EST (?),15.0,32.296705,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.635201,-77.385300,35.856300,-77.891899,Pitt Greenville Airport,Greenville,NC,US,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,night,night
3,N640EC,2021-03-10,EC35,KIXA,"Roanoke Rapids, NC",KPGV,Pitt-Greenville (KPGV),11:33PM EST,12:02AM EST (+1),28.0,54.850847,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",36.403840,-77.637860,35.635201,-77.385300,Halifax-Northampton Regional Airport,Roanoke Rapids,NC,US,Pitt Greenville Airport,Greenville,NC,US,night,night
4,N640EC,2021-03-10,EC35,KRWI,Rocky Mount-Wilson Rgnl (KRWI),KIXA,"Roanoke Rapids, NC",08:50PM EST,09:31PM EST,41.0,41.163189,"METRO AVIATION INCSHREVEPORT, LA, US(Corporation)",35.856300,-77.891899,36.413910,-77.628940,Rocky Mount Wilson Regional Airport,Rocky Mount,NC,US,Halifax-Northampton Regional Airport,Roanoke Rapids,NC,US,night,night


In [95]:
rw.isna().sum()

tail_number                 0
date                        0
aircraft                 7716
origin_code              1047
origin                      0
destination_code         1083
destination                77
departure                   0
arrival                   216
duration                  270
distance_mi               989
Owner                       0
origin_Latitude           547
origin_Longitude          547
destination_Latitude      487
destination_Longitude     487
airport_origin           1047
city_origin                77
origin_state               55
country_origin           1047
airport_destination      1083
city_destination           82
destination_state         116
country_destination      1083
departure_shift             0
arrival_shift               0
dtype: int64

In [96]:
# drop all imcomplete records
rw = rw.loc[(rw.arrival.notnull()) & (rw.duration.notnull()) & (rw.country_origin == 'US') & (rw.country_destination == 'US')\
            & (rw.duration != 0) & (rw.origin != rw.destination) & (rw.distance_mi != 0)]

In [97]:
rw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96074 entries, 0 to 136390
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   tail_number            96074 non-null  object        
 1   date                   96074 non-null  datetime64[ns]
 2   aircraft               90671 non-null  object        
 3   origin_code            96074 non-null  object        
 4   origin                 96074 non-null  object        
 5   destination_code       96074 non-null  object        
 6   destination            96074 non-null  object        
 7   departure              96074 non-null  object        
 8   arrival                96074 non-null  object        
 9   duration               96074 non-null  float64       
 10  distance_mi            96074 non-null  float64       
 11  Owner                  96074 non-null  object        
 12  origin_Latitude        96074 non-null  float64       
 13  

In [98]:
# import to csv
rw.to_csv("..//Datasets/NEW/RW_with_airports.csv", index=False)